In [29]:
# Import dependencies

import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy import inspect
import config

### Extract CSVs into Dataframe

In [30]:
movies_file = "Resources/movies.csv"
movies_df = pd.read_csv(movies_file)
movies_df.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [31]:
ratings_file = "Resources/ratings.csv"
ratings_df = pd.read_csv(ratings_file)
ratings_df.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


### Transform movies Dataframe

In [32]:
# Create a filtered dataframe from specific columns
movies_cols = ["movieId","title","genres"]
movies_transformed = movies_df[movies_cols].copy()

# Rename the column headers
movies_transformed= movies_transformed.rename(columns={
                                                          "movieId": "movie_id"
                                                          })

# Clean the data by dropping duplicates and NA values and setting the index
movies_transformed.drop_duplicates("movie_id", inplace=True)
movies_transformed.head()

,movie_id,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


### Transform ratings Dataframe

In [33]:
# Create a filtered dataframe from specific columns
ratings_cols = ["userId","movieId","rating","timestamp"]
ratings_transformed = ratings_df[ratings_cols].copy()

# Rename the column headers
ratings_transformed= ratings_transformed.rename(columns={
                                                          "movieId": "movie_id",
                                                          "userId" : "user_id",
                                                          "timestamp" : "timestamps"
                                                          })

# Clean the data by dropping duplicates and NA values and setting the index
ratings_transformed.drop_duplicates("movie_id", inplace=True)
ratings_transformed.head()

,user_id,movie_id,rating,timestamps
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


### Connect to Local Database

In [34]:
protocol = 'postgresql'
username = config.Username
password = config.Password
host = 'localhost'
port = 5432
database_name = 'movies_db'
rds_connection_string = f'{protocol}://{username}:{password}@{host}:{port}/{database_name}'
engine = create_engine(rds_connection_string)
insp = inspect(engine)

### Inspect Tables

In [35]:
insp.get_table_names()

['movies', 'ratings']

### Load

In [36]:
movies_transformed.to_sql(name='movies', con=engine, if_exists='append', index=False)

IntegrityError: (psycopg2.errors.UniqueViolation) duplicate key value violates unique constraint "movies_pkey"
DETAIL:  Key (movie_id)=(1) already exists.

[SQL: INSERT INTO movies (movie_id, title, genres) VALUES (%(movie_id)s, %(title)s, %(genres)s)]
[parameters: ({'movie_id': 1, 'title': 'Toy Story (1995)', 'genres': 'Adventure|Animation|Children|Comedy|Fantasy'}, {'movie_id': 2, 'title': 'Jumanji (1995)', 'genres': 'Adventure|Children|Fantasy'}, {'movie_id': 3, 'title': 'Grumpier Old Men (1995)', 'genres': 'Comedy|Romance'}, {'movie_id': 4, 'title': 'Waiting to Exhale (1995)', 'genres': 'Comedy|Drama|Romance'}, {'movie_id': 5, 'title': 'Father of the Bride Part II (1995)', 'genres': 'Comedy'}, {'movie_id': 6, 'title': 'Heat (1995)', 'genres': 'Action|Crime|Thriller'}, {'movie_id': 7, 'title': 'Sabrina (1995)', 'genres': 'Comedy|Romance'}, {'movie_id': 8, 'title': 'Tom and Huck (1995)', 'genres': 'Adventure|Children'}  ... displaying 10 of 9742 total bound parameter sets ...  {'movi

In [37]:
ratings_transformed.to_sql(name='ratings', con=engine, if_exists='append', index=False)

### Confirm data has been added by querying the movies table

In [38]:
pd.read_sql_query('select * from movies', con=engine).head()

,movie_id,title,genres
0,1,Toy Story (1995) ...,Adventure|Animation|Children|Comedy|Fantasy ...
1,2,Jumanji (1995) ...,Adventure|Children|Fantasy ...
2,3,Grumpier Old Men (1995) ...,Comedy|Romance ...
3,4,Waiting to Exhale (1995) ...,Comedy|Drama|Romance ...
4,5,Father of the Bride Part II (1995) ...,Comedy ...


### Confirm data has been added by querying the ratings table

In [39]:
pd.read_sql_query('select * from ratings', con=engine).head()

,user_id,movie_id,rating,timestamps
0,1,1,4,964982703
1,1,3,4,964981247
2,1,6,4,964982224
3,1,47,5,964983815
4,1,50,5,964982931


### Join Data from movies and ratings

In [41]:
sql_join = r"""SELECT movies.movie_id, movies.title, movies.genres, ratings.user_id, ratings.rating, ratings.timestamps
FROM movies
INNER JOIN ratings
ON movies.movie_id=ratings.movie_id;"""
pd.read_sql_query(sql_join, con=engine)

,movie_id,title,genres,user_id,rating,timestamps
0,1,Toy Story (1995) ...,Adventure|Animation|Children|Comedy|Fantasy ...,1,4,964982703
1,1,Toy Story (1995) ...,Adventure|Animation|Children|Comedy|Fantasy ...,1,4,964982703
2,2,Jumanji (1995) ...,Adventure|Children|Fantasy ...,6,4,845553522
3,2,Jumanji (1995) ...,Adventure|Children|Fantasy ...,6,4,845553522
4,3,Grumpier Old Men (1995) ...,Comedy|Romance ...,1,4,964981247
...,...,...,...,...,...,...
19443,193585,Flint (2017) ...,Drama ...,184,4,1537109805
19444,193587,Bungo Stray Dogs: Dead Apple (2018) ...,Action|Animation ...,184,4,1537110021
19445,193587,Bungo Stray Dogs: Dead Apple (2018) ...,Action|Animation ...,184,4,1537110021
19446,193609,Andrew Dice Clay: Dice Rules (1991) ...,Comedy ...,331,4,1537157606
